# Web scraping application with Python using Requests and BeautifulSoup

### Extraction of climate data located in tables on web pages

<img src = './img/ws_requests_beautifulsoup.jpg' width = 800>

### **Consideraciones legales y éticas**

Esta publicación no trata sobre cómo extraer datos de una página web con fines ilegales.
Hay que asegurarse de tener permiso antes de extraer ciertos tipos de datos que puede violar los términos del servicio o incluso regulaciones legales:

- Revise los términos de uso de la página web en relación a los permisos de extracción de datos.
- Priorice el uso de las APIs, si están disponibles, ya que proporcionan acceso legal a los datos.
- Póngase en contacto directamente con el propietario de la página web para comprobar el permiso de extracción de datos.

### **Introducción**

En este proyecto veremos cómo se pueden extraer datos de una página web que contiene información climática en diversas estaciones meteorológicas en toda España, haciendo uso de **requests** y **BeautifulSoup**.

En concreto, la página web es de "eltiempo.es" y me centraré en la localidad de A Coruña, obteniendo una tabla de valores climáticos para los próximos 9 días.

### **Instalación de paquetes**

A continuación se muestran las librerías necesarias para la ejecución del código.

- ipykernel:  kernel de Jupyter necesario para ejecutar Python
- tqdm: herramienta útil para crear barras de progeso en bucles
- joblib: biblioteca que permite la serialización de objetos Python.
- pandas: biblioteca escrita sobre Numpy para manipulación y análisis de datos
- request: librería que permite hacer peticiones HTTP
- bs4: librería BeautifulSoup que permite extraer datos de HTML y XML


Ejecutamos en la terminal el siguiente código:

`pip install ipykernel tqdm joblib pandas requests bs4`

### **Importación de librerías**

In [1]:
import os # proporciona funcionalidad del sistema operativo
import datetime # manipulación de fechas y horas
import pandas as pd # manipulación y análisis de datos
import requests # realizar peticiones HTTP
from bs4 import BeautifulSoup # extraer datos de HTML y XML
from csv import writer # crea un objeto para escribir datos en archivos csv
from csv import DictWriter # crea un objeto para escribir datos en archivos csv donde cada fila es un diccionario

### **Funciones útiles**

Creación de funciones que utilizaremos más adelante para generar un archivo en formato csv y almacenar datos en él.

In [2]:
def append_header_as_row(file_name, field_names):
    """ 
    Abre un csv y agrega los nombres de las columnas (headers).
    Args: 
    file_name: nombre del archivo que abrimos,
    field_names: headers que añadimos como una lista.
    """

    # Compruebo si el archivo existe
    file_existe = os.path.isfile(file_name)
    
    # Si existe, se elimina
    if file_existe:
        os.remove(file_name)
        print(f'Archivo {file_name} eliminado.')       

    # Abre el archivo en modo 'append'
    with open(file_name, 'a+', newline='') as write_obj:
        # Crea un objeto 'writer' desde el módulo csv
        dict_writer = DictWriter(write_obj, fieldnames=field_names)
        # Añade la cabecera al objeto
        dict_writer.writeheader() 
        
def append_dict_as_row(file_name, dict_of_elem, field_names):
    """ 
    Abre un csv y agrega una fila en forma de diccionario en el csv.
    Args:
    file_name: nombre del csv que abrimos,
    dict_of_elem: diccionario con los elementos de la fila que se va a añadir al final del csv,
    field_names: nombres de las columnas (headers) en forma de lista.
    """

    # Abre el archivo en modo 'append'
    with open(file_name, 'a+', newline='') as write_obj:
        # Crea un objeto 'writer' desde el módulo csv
        dict_writer = DictWriter(write_obj, fieldnames=field_names)
        #  Añade un diccionario de filas al csv
        dict_writer.writerow(dict_of_elem)

### **Solicitud GET a la URL y obtención de los datos HTML**

In [3]:
# Definición de la url
keyword = 'a-coruna' # poner la localidad deseada
url = 'https://www.eltiempo.es/' + keyword +'.html'

# Solicitud GET a la url y respuesta HTML con BeautifulSoup
print('[TASK] Enviando petición GET para obtener los datos...')
page = requests.get(url)
content = page.content
soup = BeautifulSoup(content, 'html.parser')

# función prettify() para que la estructura del html se vea mejor
print(soup.prettify())
print('[OK] Respuesta HTTP recibida correctamente!')

[TASK] Enviando petición GET para obtener los datos...
<!DOCTYPE html>
<html lang="es-es">
 <head>
  <meta charset="utf-8"/>
  <!-- Always force latest IE rendering engine or request Chrome Frame -->
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport">
   <meta content="yes" name="apple-mobile-web-app-capable">
    <!-- Use title if it's in the page YAML frontmatter -->
    <title>
     El Tiempo en A Coruña, A Coruña - 14 días
    </title>
    <meta content="A Coruña, A Coruña, España" name="locality"/>
    <meta content="El Tiempo en A Coruña, A Coruña para los próximos 14 días, previsión actualizada del tiempo. Temperaturas, probabilidad de lluvias y velocidad del viento." name="description"/>
    <meta content="El Tiempo en A Coruña, A Coruña para los próximos 14 días, previsión actualizada del tiempo. Temperaturas, probabilidad de lluvias y velocidad del viento." prope

### **Inspeccionar la página web**

Al analizar la web vemos que todos los valores se encuentran en una tabla `<table>`.

<img src = './img/inspeccion_tabla.jpg' width = 800>

La tabla se encuentra dividida en dos partes: la cabezera `<thead>` y el cuerpo `<tbody>`.

**Head**

En la cabecera de la tabla podemos ver que aparece el día de la semana, la fecha, la temperatura máxima y la temperatura mínima de cada uno de los días, y todos se encuentran en una misma fila `<tr>`

<img src = './img/inspeccion_head.jpg' width = 800>

Si analizamos el contenido del `<tr>` vemos que aparecen una serie de etiquetas `<th>`. Cada una de ellas representa las columnas de la cabecera.

En cada `<th>` aparecen los datos buscados.

<img src = './img/inspeccion_columnas_head.jpg' width=800>

**Body**

En la parte del body nos encontramos con las temperaturas para 08:00, 14:00 y 20:00 horas, así como la cantidad de lluvia, de nieve, el viento, la hora en que amanece y la hora en que anochece, para cada uno de los días. Cada una de ellas se encuentra en un `<tr>` diferente del body.

Dentro de cada `<tr>` se encuentran una serie de etiquetas `<td>` que representan cada una de las columnas de la fila correspodiente. En ellas aparecen los atributos necesarios para hacer la extracción de datos.

<img src = './img/inspeccion_filas_body.jpg' width = 800>

### **Extracción de los datos**

Extraemos los datos del "head" y del "body" para poder manipular su contenido.

In [4]:
# obtener la tabla para poder tratar su contenido
print('[TASK] Analizando la tabla para obtener los valores actuales...')

# guardar los datos de la tabla indicando su clase
table = soup.find_all('table', attrs={'class': 'table'})[0]

# guardar el header de la tabla
head = table.thead
print('[OK] Valores del Head obtenidos correctamente')

# guardar el body de la tabla
body = table.tbody
print('[OK] Valores del Body obtenidos correctamente')

[TASK] Analizando la tabla para obtener los valores actuales...
[OK] Valores del Head obtenidos correctamente
[OK] Valores del Body obtenidos correctamente


Genero el archivo csv en el que voy a guardar los datos de la tabla y añado la cabecera.

In [5]:
# HEADER
keys=[keyword]

for h in range(0, 9):
    keys.append(head.find('tr').find_all(class_ = 'text-poppins-bold')[h].text.replace('\n                            ','').replace('\n                        ',''))
keys.append('Fecha_Creacion')

# genero el csv y agrego el header
print('[TASK] Añadiendo los nombres de las columnas al fichero CSV...')
append_header_as_row(
    file_name = 'clima-' + keyword + '.csv',
    field_names = keys
)
print('[OK] Valor añadido correctamente en el fichero CSV!')

[TASK] Añadiendo los nombres de las columnas al fichero CSV...
[OK] Valor añadido correctamente en el fichero CSV!


Extraemos los datos de la cabecera: día de la semana, la fecha, la temperatura máxima y la temperatura mínima de cada uno de los días y los guardamos en un archivo csv.

In [6]:
# FECHA

print('[TASK] Analizando la tabla para obtener las fechas ...')
fecha=[]

fecha.append(head.find_all('tr')[0].find_all('th')[0].text)

for f in range(1, 10):
    fecha.append(
        head.find_all('tr')[0].find_all('th')[f].find(class_='text-roboto-condensed').text.replace('\n                                ','').replace('\n                            ','')
    )
    print('[OK] Valor encontrado correctamente!')
    
fecha.append(datetime.date.today())
dictionary = dict(zip(keys,fecha))
    
# Guardamos cada una de las filas generadas en el csv
print('[TASK] Añadiendo el nuevo valor al fichero CSV...')
append_dict_as_row(
    file_name = 'clima-' + keyword + '.csv', 
    dict_of_elem = dictionary, # diccionario de valores
    field_names = keys   # nombres de las columnas     
)
print('[OK] Valor añadido correctamente en el fichero CSV!')

# TEMPERATURA MÁXIMA

print('[TASK] Analizando la tabla para obtener las temperaturas máximas ...')
tmaxima=[]

tmaxima.append(head.find_all('tr')[0].find_all('th')[0].text)

for tmax in range(1, 10):
    tmaxima.append(
        head.find_all('tr')[0].find_all('th')[tmax].find_all(class_='degrees')[0].text
    )
    print('[OK] Valor encontrado correctamente!')
    
tmaxima.append(datetime.date.today())
dictionary = dict(zip(keys,tmaxima))
    
# Guardamos cada una de las filas generadas en el csv
print('[TASK] Añadiendo el nuevo valor al fichero CSV...')
append_dict_as_row(
    file_name = 'clima-' + keyword + '.csv', 
    dict_of_elem = dictionary, # diccionario de valores
    field_names = keys   # nombres de las columnas     
)
print('[OK] Valor añadido correctamente en el fichero CSV!')

# TEMPERATURA MÍNIMA

print('[TASK] Analizando la tabla para obtener las temperaturas mínimas ...')

tminima=[]

tminima.append(head.find_all('tr')[0].find_all('th')[0].text)

for tmin in range(1, 10):
    tminima.append(
        head.find_all('tr')[0].find_all('th')[tmin].find_all(class_='degrees')[1].text
    )
    print('[OK] Valor encontrado correctamente!')
    
tminima.append(datetime.date.today())
dictionary = dict(zip(keys,tminima))
    
# Guardamos cada una de las filas generadas en el csv
print('[TASK] Añadiendo el nuevo valor al fichero CSV...')
append_dict_as_row(
    file_name = 'clima-' + keyword + '.csv', 
    dict_of_elem = dictionary, # diccionario de valores
    field_names = keys   # nombres de las columnas     
)
print('[OK] Valor añadido correctamente en el fichero CSV!')

[TASK] Analizando la tabla para obtener las fechas ...
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[TASK] Añadiendo el nuevo valor al fichero CSV...
[OK] Valor añadido correctamente en el fichero CSV!
[TASK] Analizando la tabla para obtener las temperaturas máximas ...
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[OK] Valor encontrado correctamente!
[TASK] Añadiendo el nuevo valor al fichero CSV...
[OK] Valor añadido correctamente en el fichero CSV!
[TASK]

Extraemos los datos del body: temperaturas a las 08:00, 14:00 y 20:00; las precipitaciones de lluvia y nieve; la velocidad del viento, la hora en que amanece y en que anochece y los guardamos en el archivo csv.

In [7]:
# TEMPERATURAS A LAS 08:00, 14:00 Y 20:00
# cada 'tr' representa una fila
# cada 'td' representa una columna

print('[TASK] Analizando la tabla para obtener las temperaturas ...')

for row in range(0,3):

    temperaturas=[]

    temperaturas.append(body.find_all('tr')[row].find_all('td')[0].text)
    try:
        temperaturas.append(body.find_all('tr')[row].find_all('td')[1].find_all(class_='degrees')[0].text) # segunda columna
    except:
        temperaturas.append(body.find_all('tr')[row].find_all('td')[1].text.replace('\n',''))

    for col in range(2,10):
        temperaturas.append(body.find_all('tr')[row].find_all('td')[col].find_all(class_='degrees')[0].text.replace('\n',''))

    temperaturas.append(datetime.date.today())

    print('[OK] Valor encontrado correctamente!')
        
    # creamos el diccionario que exportaremos al csv
    dictionary = dict(zip(keys,temperaturas))
        
    # Guardamos cada una de las filas generadas en el csv
    print('[TASK] Añadiendo el nuevo valor al fichero CSV...')
    append_dict_as_row(
        file_name = 'clima-' + keyword + '.csv', 
        dict_of_elem = dictionary, # diccionario de valores
        field_names = keys   # nombres de las columnas     
    )
    print('[OK] Valor añadido correctamente en el fichero CSV!')

# LLUVIA Y NIEVE

print('[TASK] Analizando la tabla para obtener las precipitaciones de lluvia y nieve ...')

for rain in range(3,5):
    
    lluvia = []
    
    for col in range(0, 10):
        lluvia.append(body.find_all('tr')[rain].find_all('td')[col].text)
        
    lluvia.append(datetime.date.today())

    print('[OK] Valor encontrado correctamente!')
        
    # creamos el diccionario que exportaremos al csv
    dictionary = dict(zip(keys,lluvia))
        
    # Guardamos cada una de las filas generadas en el csv
    print('[TASK] Añadiendo el nuevo valor al fichero CSV...')
    append_dict_as_row(
        file_name = 'clima-' + keyword + '.csv', 
        dict_of_elem = dictionary, # diccionario de valores
        field_names = keys   # nombres de las columnas     
    )
    print('[OK] Valor añadido correctamente en el fichero CSV!')  
    
# VIENTO

print('[TASK] Analizando la tabla para obtener la velocidad del viento ...')

viento = []

viento.append(body.find_all('tr')[5].find_all('td')[0].text)

for col in range(1,10):
    viento.append(body.find_all('tr')[5].find_all('td')[col].find(class_='wind-text-value').text + ' ' + body.find_all('tr')[5].find_all('td')[col].find(class_='wind-text-unit').text)
    
viento.append(datetime.date.today())

print('[OK] Valor encontrado correctamente!')
        
# creamos el diccionario que exportaremos al csv
dictionary = dict(zip(keys,viento))
        
# Guardamos cada una de las filas generadas en el csv
print('[TASK] Añadiendo el nuevo valor al fichero CSV...')
append_dict_as_row(
    file_name = 'clima-' + keyword + '.csv', 
    dict_of_elem = dictionary, # diccionario de valores
    field_names = keys   # nombres de las columnas     
)
print('[OK] Valor añadido correctamente en el fichero CSV!')

# HORA AMANECER Y ANOCHECER

print('[TASK] Analizando la tabla para obtener las horas de amanecer y anochecer ...')

for sol in range(6,8):
    
    luz = []
    
    for col in range(0, 10):
        luz.append(body.find_all('tr')[sol].find_all('td')[col].text)
        
    luz.append(datetime.date.today())

    print('[OK] Valor encontrado correctamente!')
        
    # creamos el diccionario que exportaremos al csv
    dictionary = dict(zip(keys,luz))
        
    # Guardamos cada una de las filas generadas en el csv
    print('[TASK] Añadiendo el nuevo valor al fichero CSV...')
    append_dict_as_row(
        file_name = 'clima-' + keyword + '.csv', 
        dict_of_elem = dictionary, # diccionario de valores
        field_names = keys   # nombres de las columnas     
    )
    print('[OK] Valor añadido correctamente en el fichero CSV!')  

[TASK] Analizando la tabla para obtener las temperaturas ...
[OK] Valor encontrado correctamente!
[TASK] Añadiendo el nuevo valor al fichero CSV...
[OK] Valor añadido correctamente en el fichero CSV!
[OK] Valor encontrado correctamente!
[TASK] Añadiendo el nuevo valor al fichero CSV...
[OK] Valor añadido correctamente en el fichero CSV!
[OK] Valor encontrado correctamente!
[TASK] Añadiendo el nuevo valor al fichero CSV...
[OK] Valor añadido correctamente en el fichero CSV!
[TASK] Analizando la tabla para obtener las precipitaciones de lluvia y nieve ...
[OK] Valor encontrado correctamente!
[TASK] Añadiendo el nuevo valor al fichero CSV...
[OK] Valor añadido correctamente en el fichero CSV!
[OK] Valor encontrado correctamente!
[TASK] Añadiendo el nuevo valor al fichero CSV...
[OK] Valor añadido correctamente en el fichero CSV!
[TASK] Analizando la tabla para obtener la velocidad del viento ...
[OK] Valor encontrado correctamente!
[TASK] Añadiendo el nuevo valor al fichero CSV...
[OK] Va

### **Cargamos el csv en un DataFrame**

Utilizando la librería de Pandas, cargamos el archivo csv creado en un DataFrame para poder visualizar la tabla con los datos obtenidos.

In [8]:
df = pd.read_csv('clima-' + keyword + '.csv')

# Poner nombre a las filas incompletas
df.iloc[0,0] = 'Fecha'
df.iloc[1,0] = 'Tmax'
df.iloc[2,0] = 'Tmin'
df.iloc[9,0] = 'Amanecer'
df.iloc[10,0] = 'Anochecer'
df

,a-coruna,Hoy,Mañ,Lun,Mar,Mié,Jue,Vie,Sáb,Dom,Fecha_Creacion
0,Fecha,7 SEP,8 SEP,9 SEP,10 SEP,11 SEP,12 SEP,13 SEP,14 SEP,15 SEP,2024-09-07
1,Tmax,21°,21°,20°,20°,20°,19°,19°,21°,21°,2024-09-07
2,Tmin,14°,14°,16°,15°,15°,16°,15°,14°,15°,2024-09-07
3,08:00,NaN,16°,18°,17°,15°,17°,15°,14°,15°,2024-09-07
4,14:00,21°,21°,20°,20°,20°,19°,19°,21°,21°,2024-09-07
5,20:00,19°,20°,19°,18°,18°,17°,17°,19°,19°,2024-09-07
6,Lluvia,0 mm,1.1 mm,0.4 mm,0 mm,2.2 mm,4.4 mm,0 mm,0 mm,0 mm,2024-09-07
7,Nieve,0 cm,0 cm,0 cm,0 cm,0 cm,0 cm,0 cm,0 cm,0 cm,2024-09-07
8,Viento,14 km/h,14 km/h,17 km/h,22 km/h,14 km/h,14 km/h,15 km/h,14 km/h,14 km/h,2024-09-07
9,Amanecer,08:03,08:04,08:05,08:06,08:07,08:08,08:10,08:11,08:12,2024-09-07


Como podéis apreciar hay un NaN a las 08:00 horas del día de Hoy; eso es debido a que el código se ejecutó entre las 08:00 y las 14:00 horas. Si el código se ejecuta entre las 14:00 y las 20:00 horas, aparecerá un NaN también a las 14:00 horas. Si el código se ejecuta después de las 20:00 horas, la columna "Hoy" ya no estará disponible y empezará la tabla en "Mañana".

### **Conclusión**

Una de las combinaciones más comunes en web scraping es usar "requests" para obtener el contenido HTML de una página web y luego usar "BeautifulSoup" para analizar y extraer la información deseada de ese contenido. Ambas bibliotecas juntas son muy poderosas para realizar dichas tareas.

En este artículo hemos visto cómo podemos extraer los datos climáticos de los próximos 9 días de una localidad de España, aunque podríamos utilizar el mismo código para obtener los de cualquier localidad, simplemente con cambiar el contenido de la variable "keyword".

### **Referencias**

- [Python](https://www.python.org/)
- [Ipykernel](https://pypi.org/project/ipykernel/)
- [tqdm](https://pypi.org/project/tqdm/)
- [joblib](https://pypi.org/project/joblib/)
- [Pandas](https://pandas.pydata.org/)
- [Requests](https://pypi.org/project/requests/)
- [BeautifulSoup](https://pypi.org/project/beautifulsoup4/)